In [ ]:
import datetime as dt
import pathlib

import geopandas as gpd
import google.auth
import numpy as np
import pandas as pd
from gtfslite import GTFS
from shared_utils import catalog_utils, gtfs_utils_v2, rt_dates

In [ ]:
from retrospective_feed_generation import *
from warehouse_utils import *
from gtfs_utils import *

### Get RT Data

In [ ]:
CREDENTIALS, _ = google.auth.default()

# not used
def safe_read_geoparquet(*args, **kwargs):
    assert "storage_options" not in kwargs
    return gpd.read_parquet(
        *args, **kwargs, storage_options={"token": CREDENTIALS.token}
    )

In [ ]:
SAMPLE_DATE_STR = rt_dates.DATES["apr2025"]
FEED_NAME = "Big Blue Bus Schedule"

In [ ]:
SAMPLE_DATE_STR

In [ ]:
feed_key = (
    gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(
        selected_date=SAMPLE_DATE_STR, keep_cols=["name", "gtfs_dataset_key"]
    )
    .set_index("name")
    .at["Big Blue Bus Schedule", "gtfs_dataset_key"]
)
feed_key

In [ ]:
schedule_rt_stop_times_single_agency = get_schedule_rt_stop_times_table(
    feed_key,
    SAMPLE_DATE_STR
)

### Get schedule feed

In [ ]:
#TODO: right now this was just a download based on the url in airtable
# Need to make it traceable instead
GTFS_FEED_PARENT = "../conveyal_update/feeds_2025-04-16/socal/Big_Blue_Bus_Schedule_7a3f513c343b16a30c135ed7d332b6d6_gtfs.zip/"
GTFS_FEED_GLOB = "*.zip"

ARBITRARY_SERVICE_ID = "0"
GTFS_DATE_STRFTIME_CODE = "%Y%m%d"

In [ ]:
feed_paths = pathlib.Path(GTFS_FEED_PARENT).glob(GTFS_FEED_GLOB)
feed_path = next(feed_paths)
assert next(feed_paths, None) is None, "Ambiguous Schedule Feed"
feed = GTFS.load_zip(feed_path)
feed_filtered = subset_schedule_feed_to_one_date(
    feed, dt.date.fromisoformat(SAMPLE_DATE_STR)
)

### Merge schedule / rt

In [ ]:
output_feed = make_retrospective_feed_single_date(
    filtered_input_feed=feed_filtered,
    stop_times_table=schedule_rt_stop_times_single_agency,
    stop_times_desired_columns=[
        "trip_id",
        "arrival_time",
        "departure_time"
        "drop_off_type",
        "pickup_type",
        "stop_headsign",
        "stop_id",
        "stop_sequence",
    ]
)

In [ ]:
output_feed.write_zip(f"output_feeds/bbb_test_{SAMPLE_DATE_STR}.zip")

### Dropped shapes and stops

In [ ]:
print("Get dropped shapes by their frequency")
feed_filtered.trips.loc[
    ~feed_filtered.trips.shape_id.isin(output_feed.trips.shape_id.unique()), "shape_id"
].value_counts()

In [ ]:
print("Get dropped stops by the number of trips serving them in the original feed")
pd.DataFrame(
    feed_filtered.stop_times.loc[
        ~feed_filtered.stop_times.stop_id.isin(
            output_feed.stop_times.stop_id.unique()
        ),
        "stop_id",
    ]
    .value_counts()
    .rename("stop_count")
).merge(
    feed_filtered.stops.set_index("stop_id")["stop_name"],
    how="left",
    left_index=True,
    right_index=True,
)

### Sample Trip

In [ ]:
output_feed.stop_times.loc[output_feed.stop_times["trip_id"] == "902110"].sort_values(
    "stop_sequence"
)

In [ ]:
feed_filtered.stop_times.loc[
    feed_filtered.stop_times["trip_id"] == "902110"
].sort_values("stop_sequence")